## Import libraries

In [1]:
import numpy as np
import pandas as pd
import os
import json
import zipfile
import gc
import matplotlib.pylab as plt
import seaborn as sns
import warnings 
import datetime
from sklearn.preprocessing import LabelEncoder
sns.set_style("whitegrid")
from tqdm import tqdm
warnings.filterwarnings('ignore')
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
import pickle
from prettytable import PrettyTable
import prettytable
from functools import reduce
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from IPython.display import Image
import lightgbm
import pickle
import xgboost as xgb
from scipy.stats import randint as sp_randint

master_path = '/content/drive/MyDrive/Elo_kaggle/'

## Loading Functions

In [2]:
## Reference: https://www.kaggle.com/c/champs-scalar-coupling/discussion/96655

def reduce_memory_usage(df, verbose=True):
  '''
  This function reduces the memory sizes of datafram by changing the dattypes of the columns.
  Parameters
  df - DataFrame whose size to be reduced
  verbose - Boolean, to mention the verbose required or not.
  '''
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

In [3]:
def downloadData():
  ## creating a dictionary with username and key
  api_token = {"username":"praveenjalaja","key":"254009cd9fcebc728fb2b82a4f00d301"}
  !mkdir -p ~/.kaggle
  ## new json created for download the data 
  with open('kaggle.json', 'w') as file:
      json.dump(api_token, file)
  !cp kaggle.json ~/.kaggle/
  !ls ~/.kaggle
  !chmod 600 /root/.kaggle/kaggle.json

  ## download the neccesary data
  !kaggle competitions download -c elo-merchant-category-recommendation

  ! unzip /content/historical_transactions.csv.zip
  !rm /content/historical_transactions.csv.zip
  ! unzip /content/new_merchant_transactions.csv.zip
  !rm /content/new_merchant_transactions.csv.zip
  ! unzip /content/merchants.csv.zip
  !rm /content/merchants.csv.zip
  ! unzip /content/train.csv.zip
  !rm /content/train.csv.zip
  ! unzip /content/test.csv.zip
  !rm /content/test.csv.zip


In [4]:
def loadData():


  train_df                   = pd.read_csv('/content/train.csv', parse_dates=["first_active_month"])
  test_df = pd.read_csv('/content/test.csv', parse_dates=["first_active_month"])
  historical_transactions_df   = pd.read_csv('/content/historical_transactions.csv',parse_dates=['purchase_date'])
  category_3_encode(historical_transactions_df)
  ###merchants_df                 = pd.read_csv('/content/merchants.csv')
  new_merchant_transactions_df = pd.read_csv('/content/new_merchant_transactions.csv',parse_dates=["purchase_date"])
  category_3_encode(new_merchant_transactions_df)
  ##historical_transactions_df = reduce_memory_usage(historical_transactions_df)
  ##new_merchant_transactions_df = reduce_memory_usage(new_merchant_transactions_df)
  return train_df,historical_transactions_df,new_merchant_transactions_df,test_df


In [5]:
def aggregated_features(new_df, df, aggs, grpby, name='',  prefix='', use_col=False):
  '''
  This function is to find the 
  aggregated values (min,max,mean,sum,nunique,std) for a columns aggregated by the groupby operation
  
  Parameters:
  new_df   - features will be added to this DF
  df       - original DF from which the features will be created
  grpby    - based on this column we'll to group by
  name     - name for the new features created
  aggs     - dictionary contains key as the column the operation performed and list of operations as the value.
  prefix   - added to the name of the feature -- default value empty
  use_col  - if set True then the original column name will be uesd to name the new feature -- default value False
  '''
  ## boolean for using the original column name in the aggregated features
  ## iterating through the columns of the need to be aggregated 

  for col, funcs in aggs.items():
    for func in funcs:
        # Getting the name of aggregation function
        if isinstance(func, str):
            func_str = func
        else:
            func_str = func.__name__ 
        # create the column
        if use_col:
          name = prefix+'_'+col+'_'+'{}'.format(func_str)

        new_df[name] = df.groupby([grpby])[col].agg(func).values

  return new_df 

In [6]:
def label_encoder(df, cols):
  '''
  This Function label encode the values in the specified columns and 
  return the data frame
  Parameters:
  df   - Original DataFrame
  cols - label encode the specified columns
  '''
  lbl_enc = LabelEncoder()
  for col in cols:
    df[col] = lbl_enc.fit_transform(df[col].astype(str))
  return df

In [7]:
def category_3_encode(df):

    ## label encode the category 3 variables.
  d = {'A':1, 'B':2, 'C':3}
  df['category_3']= df['category_3'].map(d)


In [8]:
def encode_transactions(df):
  '''This function is specially for encode the categorical values of 
  transactions data
  parameters:
  df: the Dataframe where the label encoding will performed on certain features
  '''


  #label encode the variables
  df = label_encoder(df, ['authorized_flag','category_1'])
  return df


In [9]:
## Reference: https://medium.com/towards-artificial-intelligence/handling-missing-data-for-advanced-machine-learning-b6eb89050357

def imputation_models(df, non_categorical,model_names, filepath, format):
  '''This function is an universal function for imputation using 
  ML models
  parameters:
  df : The DataFrame
  non_categorical : List of non_categorical columns names which can't be used for prediction model
  nan_features: List of feature names which contains nan_values
  models: dict , Key: feature name , value: Model
  model_names: dict, key: feature name, value: name to be saved
  filepath: file path where the models are saved.
  format: format of model to be saved like '.sav'.
  '''

  if df[df.keys()].isna().any().any():
    ## collecting all the non-categorical features 
    a = df[non_categorical]

    ## dropping all the above collected features
    df.drop(non_categorical, axis=1, inplace=True)
    gc.collect()

    #select only columns which doesn't have any null values.
    no_nan = [c for c in df.columns if c not in model_names.keys()]

    for feat in model_names.keys():

      if df[feat].isna().any().any():

        #test set by selecting only rows which are having null values
        df_null = df[df[feat].isna()]

        #train set by selecting rows which doesn't have any null values
        df_train = df.dropna()

        ##after getting the required features, 
        ##we have to build a model with no-null observations to predict the null observstions.

        file_directory = filepath+model_names[feat]+'.'+format

        clf = pickle.load(open(file_directory, 'rb'))
          

        #make prediction only for the rows with null value
        df.loc[df[feat].isna(), feat] = clf.predict(df_null[no_nan])

    df[non_categorical] = a[non_categorical]
    del a

  return df

In [10]:
def oneHotEncoding(df, features, original_df):
  '''This function is for one-hot encoding the categorical features
  parameters:
  df: DataFrame
  features: Features needs to be one hot encoded.'''

  for feat in features:
    unique_values = original_df[feat].unique()

    for cat in unique_values:
      df[feat+'={}'.format(cat)] = (df[feat] == cat).astype(int)


In [11]:


def get_monthlag_stat(new_df, df, grpby, op, col, name, prefix=''):
  
  '''
  Thid function is group by the the specified column and find the count or sum depending on the input.
  Then perform basic operations like std, min, max etcetera
  parameters
  new_df - new features will be added to this DF
  df     - original DF
  grpby  - column using which we will group the data by
  col    - operations will be performed on this column
  name   - name for this columnn
  prefix - prefix to the column name
  '''
  if op == 'sum':
    tmp = df.groupby(grpby)[col].sum().unstack()
    new_df[prefix+grpby[1]+'_'+name[0]] = tmp.reset_index().iloc[:, -1].values
    new_df[prefix+grpby[1]+'_'+name[1]] = tmp.reset_index().iloc[:, -2].values
  
  if op == 'count':
    tmp = df.groupby(grpby)[col].count().unstack()
    # check if there is any null value and fill it with 0
    if tmp.isna().sum().any() > 0:
      tmp = tmp.fillna(0.0)
    new_df[prefix+grpby[1]+'_'+name[0]] = tmp.reset_index().iloc[:, 1:].std(axis=1).values
    new_df[prefix+grpby[1]+'_'+name[1]] = tmp.reset_index().iloc[:, 1:].max(axis=1).values
    
  return new_df

In [12]:
#https://www.kaggle.com/fabiendaniel/elo-world?scriptVersionId=8335387
def successive_aggregates(df, field1, field2):
  '''
  This function does is that it group the data twice and find
  basic aggregate values.
  First it will goup by card_id and all the specified column one by one.
  Then it will find the agg values like mean, min, max and std
  for the purchase amount for each group.
  Parameters:
  df      - original DataFrame
  field1  - first groupby along with card_id
  field2  - second grouby along with card_id
  '''
  t = df.groupby(['card_id', field1])[field2].mean()
  u = pd.DataFrame(t).reset_index().groupby('card_id')[field2].agg(['mean', 'min', 'max', 'std'])
  u.columns = [field1 + '_' + field2 + '_' + col for col in u.columns.values]
  u.reset_index(inplace=True)
  return u


def successive_aggregation(new_df, df, field1='', field2 = '', other_columns=[]):
  '''
  This function is aggregates transactions columns successively with the columns mentioned.

  parameters:

  new_df  - The DataFrame which need to merged at end.
  df      - The DataFrame with all the column required.
  field1  - This is first field of many successive fields.
  field2 - The second field for the aggregation, which constant for which te aggregation done with other features.
  other_columns - all the other columns which will be later aggregated as the field_1.
  '''

  ## temp df for storing the aggregations 
  succ_agg = successive_aggregates(df, field1= field1, field2= field2)
  ## creating other successive features 
  for  col in other_columns:
    succ_agg = succ_agg.merge(successive_aggregates(df, col, field2),on=['card_id'], how='left')
  
  ## merge with the new df finally
  new_df = new_df.merge(succ_agg, on = ['card_id'], how = 'left')

  return new_df

In [13]:
def getdatefeatures(df):
  '''This function is to get all the numerical 
  features from the purchase Date feature
  Parameters:
  df: The dataFrame'''

  df['week'] = df['purchase_date'].dt.week.values
  df['dayofweek'] = df['purchase_date'].dt.dayofweek.values
  df['hour'] = df['purchase_date'].dt.hour.values


In [14]:
def timebtwpurchases(df, groupby, column, shift = 1):

  ''' This function is to extract the shifted columns between the purchases.

  parameters:

  df - The DataFrame
  groupby - The col by which the data is grouped by
  column - column for which the shifted features are calcuated
  shift - how many time periods shift is required
  '''


  ## first to sort values by the column for which shifting needed.

  df = df.sort_values(column)

  for i in range(shift):
    ## creating the shited time between two purchases in a group
    df['prev_{}_'.format(i+1)+column] = df.groupby([groupby])[column].shift(i+1)
    ## calcuating the shift in different time representations like days,seconds,hours
    df['purchase_date_diff_{}_days'.format(i+1)] = (df[column] - df['prev_{}_'.format(i+1)+column]).dt.days.values
    df['purchase_date_diff_{}_seconds'.format(i+1)] = df['purchase_date_diff_{}_days'.format(i+1)].values * 24 * 3600
    df['purchase_date_diff_{}_seconds'.format(i+1)] += (df[column] - df['prev_{}_'.format(i+1)+column]).dt.seconds.values
    df['purchase_date_diff_{}_hours'.format(i+1)] = df.iloc[:, -1].values // 3600

  return df

In [15]:
## reference: https://towardsdatascience.com/find-your-best-customers-with-customer-segmentation-in-python-61d602f9eee6
def RScore(x,p,d):
    if x <= d[p][0.011]:
        return 1
    elif x <= d[p][0.050]:
        return 2
    elif x <= d[p][0.25]: 
        return 3
    elif x <= d[p][0.5]:
        return 4
    elif x <= d[p][0.75]:
        return 5
    elif x <= d[p][0.95]:
        return 6
    elif x <= d[p][0.989]:
        return 7
    else:
        return 8
    
def FMScore(x,p,d):
    if x <= d[p][0.011]:
        return 8
    elif x <= d[p][0.050]:
        return 7
    elif x <= d[p][0.25]: 
        return 6
    elif x <= d[p][0.5]:
        return 5
    elif x <= d[p][0.75]:
        return 4
    elif x <= d[p][0.95]:
        return 3
    elif x <= d[p][0.989]:
        return 2
    else:
        return 1

In [16]:
def rfm(df,quantiles, transc):
  '''This function is to calcualte RFM score and RFM index for the dataframe with RFM.

  parameters:

  df: The DataFrame
  quantiles: qunatiles for the RFM score and index to calculated
  transc : type of transactions(new or hist)

  '''

  ## grouping quantiles
  df[transc+'r_quantile'] = df[transc+'purchase_recency'].apply(RScore, args=(transc+'purchase_recency',quantiles))
  df[transc+'f_quantile'] = df[transc+'count'].apply(FMScore, args=(transc+'count',quantiles))
  df[transc+'m_quantile'] = df[transc+'purchase_amount_sum'].apply(FMScore, args=(transc+'purchase_amount_sum',quantiles))
  ## calaculating RFM index and RFMScore
  df[transc+'RFMindex'] = df[transc+'r_quantile'].map(str)+df[transc+'f_quantile'].map(str)+df[transc+'m_quantile'].map(str)
  df[transc+'RFMindex'] = df[transc+'RFMindex'].astype(int)                     
  df[transc+'RFMScore'] = df[transc+'r_quantile']+df[transc+'f_quantile']+df[transc+'m_quantile'] 

In [17]:
holidays = {'EasterDay_2017' : '2017-04-16',
          'AllSoulsDay_2017': '2017-11-2',
          'ChristmasDay_2017': '2017-12-25',
          'FathersDay_2017': '2017-08-13',
          'ChildrenDay_2017':'2017-10-12',
          'BlackFriday_2017':'2017-11-24',
          'ValentineDay_2017':'2017-06-12',
          'MothersDay_2018':'2018-05-13'}

In [18]:
def preprocess(new_merchant_transactions,historical_transactions):
    

  non_categorical= ['card_id', 'merchant_id', 'purchase_date']


  model_names_transacations= {'category_2':'category_2_new_merchants_model',
                        'category_3':'category_3_new_merchants_model' }


  ## encoding the categorical features in new_merchants
  new_merchant_transactions = encode_transactions(new_merchant_transactions)

  new_merchant_transactions = imputation_models(df = new_merchant_transactions,
                    non_categorical = non_categorical,
                    model_names =model_names_transacations,
                    filepath = master_path,
                    format = '.sav')
  

  model_names_transacations= {'category_2':'category_2_historical_merchants_model',
                      'category_3':'category_3_historical_merchants_model' }


  ## encoding the categorical features in historical transactions
  historical_transactions = encode_transactions(historical_transactions)

  historical_transactions = imputation_models(df = historical_transactions,
                    non_categorical = non_categorical,
                    model_names =model_names_transacations,
                    filepath = master_path,
                    format = '.sav')
  
  
  ## One-hot encoding the categorical features
  categorical_features = ['category_2','category_3','month_lag']

  ## one-hot encoding historical transactions 
  oneHotEncoding(historical_transactions, features=categorical_features,
                original_df = historical_transactions_df)

  ## one-hot encoding new merchants transactions
  oneHotEncoding(new_merchant_transactions, features=categorical_features,
                original_df = new_merchant_transactions_df)
  

  ## preprocess the purchase_amount
  new_merchant_transactions['purchase_amount'] = np.round(new_merchant_transactions['purchase_amount'] / 0.00150265118 + 497.06, 2)
  historical_transactions['purchase_amount'] = np.round(historical_transactions['purchase_amount'] / 0.00150265118 + 497.06, 2)

  #is_weekend is a feature which purchase_date is weekend or weekday.
  new_merchant_transactions['is_weekend'] = new_merchant_transactions['purchase_date'].dt.dayofweek
  #>5 to check whether the day is sat or sunday then, if it is then assign a val 1 else 0
  new_merchant_transactions['is_weekend'] = new_merchant_transactions['is_weekend'].apply(lambda x: 1 if x >= 5 else 0).values
  historical_transactions['is_weekend'] = historical_transactions['purchase_date'].dt.dayofweek
  #>5 to check whether the day is sat or sunday then, if it is then assign a val 1 else 0
  historical_transactions['is_weekend'] = historical_transactions['is_weekend'].apply(lambda x: 1 if x >= 5 else 0).values

  reference_date = '2018-12-31'
  reference_date = pd.to_datetime(reference_date)
  ## calcuating month difference 
  new_merchant_transactions['month_diff'] = (reference_date - new_merchant_transactions['purchase_date']).dt.days // (30 + new_merchant_transactions['month_lag'])
  historical_transactions['month_diff'] = (reference_date - historical_transactions['purchase_date']).dt.days // (30 + historical_transactions['month_lag'])

  new_merchant_transactions['amount_month_ratio'] = new_merchant_transactions['purchase_amount'].values / (1.0 + new_merchant_transactions['month_diff'].values)
  historical_transactions['amount_month_ratio'] = historical_transactions['purchase_amount'].values / (1.0 + historical_transactions['month_diff'].values)

  getdatefeatures(historical_transactions)
  getdatefeatures(new_merchant_transactions)

  new_merchant_transactions = timebtwpurchases(new_merchant_transactions, 'card_id', 'purchase_date', 2)


  ## we are gonna represent number days as the feature. if the values is above 75 then it will become zero.
  for day, date in holidays.items():
    ## new_transactions 
    new_merchant_transactions[day] = (pd.to_datetime(date) - new_merchant_transactions['purchase_date']).dt.days
    new_merchant_transactions[day] = new_merchant_transactions[day].apply(lambda x: x if x > 0 and x < 75 else 0)


  return new_merchant_transactions,historical_transactions

In [19]:
def engineered_features(new,ht):
  
  ## new_transactions
  new_merch_features = pd.DataFrame(new.groupby(['card_id']).size()).reset_index()
  new_merch_features.columns = ['card_id', 'new_transc_count']
  ## historical_transactions 
  historical_trans_features = pd.DataFrame(ht.groupby(['card_id']).size()).reset_index()
  historical_trans_features.columns = ['card_id', 'hist_transc_count']


  ##unique id's in the transactions
  aggs = {'city_id':['nunique'],
        'state_id' :['nunique'],
        'merchant_category_id':['nunique'],
        'subsector_id':['nunique'],
        'merchant_id':['nunique']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)
  
  ## categorty enigneered features
  aggs = {'category_1':['sum', 'mean'],
          'authorized_flag': ['sum', 'mean']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)
    
  ## category_1
  new_merch_features['new_transc_category_1_sum_0'] = new_merch_features['new_transc_count'].values - \
  new_merch_features['new_transc_category_1_sum'].values
  historical_trans_features['hist_transc_category_1_sum_0'] = historical_trans_features['hist_transc_count'].values - \
  historical_trans_features['hist_transc_category_1_sum'].values
  ## authorized_flag
  new_merch_features['new_transc_denied_count'] = new_merch_features['new_transc_count'].values - \
  new_merch_features['new_transc_authorized_flag_sum'].values
  historical_trans_features['hist_transc_denied_count'] = historical_trans_features['hist_transc_count'].values - \
  historical_trans_features['hist_transc_authorized_flag_sum'].values


  ### installment features 
  aggs = {'installments':['mean', 'sum', 'max', 'min', 'std', 'skew']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)


  ### category_2 one-hot encoded features 


  aggs = {'category_2=1.0':['sum', 'mean'],
          'category_2=3.0':['sum', 'mean'],
          'category_2=2.0':['sum', 'mean'],
          'category_2=4.0':['sum', 'mean'],
          'category_2=5.0':['sum', 'mean']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)


  ### category_3 one-hot encoded features 

  aggs = {'category_3=2.0':['sum', 'mean'],
          'category_3=1.0':['sum', 'mean'],
          'category_3=3.0':['sum', 'mean']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)

  #find mean of the count of the transac for merchant id
  historical_trans_features['hist_transc_merchant_id_count_mean'] = historical_trans_features['hist_transc_count'].values / (1.0+historical_trans_features['hist_transc_merchant_id_nunique'].values)


  new_merch_features['new_transc_merchant_id_count_mean'] = new_merch_features['new_transc_count'].values/ (1.0+ new_merch_features['new_transc_merchant_id_nunique'].values)


  grpby_lag = ['card_id', 'month_lag']
  historical_trans_features = get_monthlag_stat(historical_trans_features, ht, grpby=grpby_lag, op='count',
                                            col='purchase_amount', prefix='hist_transc_', name=['count_std','count_max'])

  new_merch_features = get_monthlag_stat(new_merch_features, new, grpby=grpby_lag, op='count',
                                        col='purchase_amount', prefix='new_transc_', name=['count_std','count_max'])


  aggs = {'purchase_amount':['sum', 'mean', 'max', 'min', 'median', 'std', 'skew']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)

  ## difference in the amount spend with cards
  new_merch_features['new_transc_amount_diff'] = new_merch_features['new_transc_purchase_amount_max'].values - new_merch_features['new_transc_purchase_amount_min'].values

  historical_trans_features['hist_transc_amount_diff'] = historical_trans_features['hist_transc_purchase_amount_max'].values - historical_trans_features['hist_transc_purchase_amount_min'].values


  new_merch_features = successive_aggregation(new_merch_features, new,
                                              field1='category_1', field2 = 'purchase_amount',
                                              other_columns = ['installments', 'city_id', 
                                                              'merchant_category_id', 'merchant_id',
                                                              'subsector_id','category_2','category_3'])



  historical_trans_features = successive_aggregation(historical_trans_features, ht,
                                              field1='category_1', field2 = 'purchase_amount',
                                              other_columns = ['installments', 'city_id', 
                                                              'merchant_category_id', 'merchant_id',
                                                              'subsector_id','category_2','category_3'])


  aggs = {'month_lag': ['nunique', 'mean', 'std', 'min', 'max', 'skew']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)


  aggs = {'purchase_date': ['max','min']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)


  ## diff in purchase_date from max to min 
  new_merch_features['new_transc_purchase_date_diff'] = (new_merch_features['new_transc_purchase_date_max'] - new_merch_features['new_transc_purchase_date_min']).dt.days.values

  ## purchase_count_ratio
  new_merch_features['new_transc_purchase_count_ratio'] = new_merch_features['new_transc_count'].values / (1.0 + new_merch_features['new_transc_purchase_date_diff'].values)

  ## diff in purchase_date from max to min 
  historical_trans_features['hist_transc_purchase_date_diff'] = (historical_trans_features['hist_transc_purchase_date_max'] - historical_trans_features['hist_transc_purchase_date_min']).dt.days.values

  ## purchase_count_ratio
  historical_trans_features['hist_transc_purchase_count_ratio'] = historical_trans_features['hist_transc_count'].values / (1.0 + historical_trans_features['hist_transc_purchase_date_diff'].values)


  ## aggregate features for is_weekend 
  aggs = {'is_weekend': ['sum','mean']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='purchase_is_weekend_', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='purchase_is_weekend_', use_col=True)


  aggs = {'month_diff': ['mean', 'min', 'max']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)

  ## aggregated features on the amount ratio and month_lag.
  aggs = {'amount_month_ratio': ['mean', 'std', 'min', 'max', 'skew'],
          'month_lag=1': ['sum','mean'],
          'month_lag=2':['sum','mean']}
  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)

  aggs = {'amount_month_ratio': ['mean', 'std', 'min', 'max', 'skew'],
          'month_lag=0': ['sum','mean'],
          'month_lag=-1':['sum','mean'],
          'month_lag=-2':['sum','mean']}
  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)


  ## month_lag ratio between two month_lags.
  new_merch_features['new_transc_month_lag_1_2_ratio'] = new_merch_features['new_transc_month_lag=1_sum'] \
                                                                    / (1.0+ new_merch_features['new_transc_month_lag=2_sum'])


  ## month_lag ratio in historical transactions

  historical_trans_features['hist_transc_month_lag_0_-1_ratio'] = historical_trans_features['hist_transc_month_lag=0_sum'] \
                                                                    / (1.0+ historical_trans_features['hist_transc_month_lag=-1_sum'])

  historical_trans_features['hist_transc_month_lag_0_-2_ratio'] = historical_trans_features['hist_transc_month_lag=0_sum'] \
                                                                    / (1.0+ historical_trans_features['hist_transc_month_lag=-2_sum'])
  tmp = historical_trans_features[['hist_transc_month_lag=0_sum','hist_transc_month_lag=-1_sum','hist_transc_month_lag=-2_sum']].sum(axis=1)

  ## ratio of the summed month lags with the transaction
  historical_trans_features['hist_transc_month_lag_sum_ratio'] = tmp / (1.0+ historical_trans_features['hist_transc_count'])


  ## aggregated features on  day, hour , week
  aggs = {'week': ['nunique', 'mean', 'min', 'max'],
          'dayofweek': ['nunique', 'mean', 'min', 'max'],
          'hour':['nunique', 'mean', 'min', 'max']}

  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)

  historical_trans_features = aggregated_features(historical_trans_features, ht,
                                                  aggs, grpby='card_id',prefix='hist_transc', use_col=True)

  ## aggregation difference in time features
  aggs = {'purchase_date_diff_1_seconds': ['mean', 'std', 'max', 'min'],
          'purchase_date_diff_1_days': ['mean', 'std', 'max', 'min'],
          'purchase_date_diff_1_hours': ['mean', 'std', 'max', 'min']}


  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)
  ## aggregation difference in time features
  aggs = {'purchase_date_diff_2_seconds': ['mean', 'std', 'max', 'min'],
          'purchase_date_diff_2_days': ['mean', 'std', 'max', 'min'],
          'purchase_date_diff_2_hours': ['mean', 'std', 'max', 'min']}


  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)


    ## aggregation of holidays
  aggs = dict(zip(holidays.keys(),[['mean'] for x in holidays.keys()]))

  new_merch_features = aggregated_features(new_merch_features, new,
                                          aggs, grpby='card_id',prefix='new_transc', use_col=True)


  return new_merch_features,historical_trans_features


In [20]:
def preprocess_train(df,new_merch_features,historical_trans_features):

  df = reduce(lambda left,right: pd.merge(left,right,on='card_id', how='left'), [df,new_merch_features,
                                                                                 historical_trans_features])
  
  reference_date = pd.to_datetime('2018-12-31')
  df['year'] = df['first_active_month'].dt.year.values
  df['month'] = df['first_active_month'].dt.month.values
  ## extracting elapsed dates 
  df['hist_transc_no_of_days'] = ( pd.to_datetime(df['hist_transc_purchase_date_max']) -  pd.to_datetime(df['hist_transc_purchase_date_max'])).dt.days
  df['new_transc_no_of_days'] = (pd.to_datetime(df['new_transc_purchase_date_max']) - pd.to_datetime(df['new_transc_purchase_date_max'])).dt.days
  ## recency of the puchases in terms of fractions
  df['hist_transc_purchase_active_diff'] = (pd.to_datetime(df['hist_transc_purchase_date_min'].astype(str).apply(lambda x: x[:7])) - df['first_active_month']).dt.days.values
  df['hist_transc_purchase_recency'] = (reference_date - pd.to_datetime(df['hist_transc_purchase_date_max']))/(24*np.timedelta64(1, 'h'))
  df['new_transc_purchase_recency'] = (reference_date - pd.to_datetime(df['new_transc_purchase_date_max']))/(24*np.timedelta64(1, 'h')) 

  df = label_encoder(df, cols = ['month','year'])

  quantiles_new = df[['new_transc_purchase_recency','new_transc_count','new_transc_purchase_amount_sum']].quantile(q=[0.011,0.05,0.25,0.5,0.75,0.95,0.989]).to_dict()
  ## quantiles of RFM with historical transactions
  quantiles_hist = df[['hist_transc_purchase_recency','hist_transc_count','hist_transc_purchase_amount_sum']].quantile(q=[0.011,0.05,0.25,0.5,0.75,0.95,0.989]).to_dict()

  rfm(df,quantiles_new,transc = 'new_transc_')
  rfm(df,quantiles_hist,transc = 'hist_transc_')

  remove_cols = ['first_active_month','new_transc_purchase_date_max',
 'new_transc_purchase_date_min','hist_transc_purchase_date_max',
 'hist_transc_purchase_date_min']

  df = df.drop(labels=remove_cols, axis = 1)

  return df
  

In [21]:
def predict_loyalty_score(X):

  ''' This function is predict the given card_id's predicted loyalty Score

  parameters:
  X : List of the card_id's

  returns Dataframe with predicted loyalty score for each card_id.'''
  

  ## training the data 

  print("Fetching the transactional and card_id data")
  train = cards.loc[cards['card_id'].isin(X)]

  historical_transactions = historical_transactions_df[historical_transactions_df['card_id'].isin(X)]
  new_merchant_transactions = new_merchant_transactions_df[new_merchant_transactions_df['card_id'].isin(X)]
  print("PreProcess the transactions data......")
  new_merchant_transactions,historical_transactions  = preprocess(new_merchant_transactions,historical_transactions)
  print("Feature Engineering the transactions data.....")
  new_merch_features,historical_trans_features = engineered_features(new_merchant_transactions,historical_transactions)
  print('preprocess the Feature Engineered Data')
  train = preprocess_train(train,new_merch_features,historical_trans_features)

  train.set_index('card_id',inplace=True)

  print("Predicting the Loyalty Score.....")
  with open(master_path+'lgbm_final.sav', 'rb') as pickle_file:
    mod = pickle.load(pickle_file)
  pred_lgb = mod.predict(train , num_iteration=mod.best_iteration)

  with open(master_path+'xgboost_final.sav', 'rb') as pickle_file:
      mod = pickle.load(pickle_file)
  pred_xgb = mod.predict(xgb.DMatrix(train[mod.feature_names]), ntree_limit=mod.best_ntree_limit+50)

  meta_data = np.vstack([pred_xgb, pred_lgb]).transpose()

  with open(master_path+'stacked_final.sav', 'rb') as pickle_file:
      mod = pickle.load(pickle_file)
  pred_stack = mod.predict(meta_data)

  predcited_target = pd.DataFrame()

  predcited_target['card_id'] = train.index 

  predcited_target['predicted_target'] = pred_stack

  predcited_target.set_index('card_id',inplace=True)

  return predcited_target

In [22]:
def rmse_score(X , target):

  pred = predict_loyalty_score(X)
  ## calculating the RMSE score
  score = np.sqrt(mean_squared_error(pred, target))
  print('RMSE Score:', score)

  return score




## Downloading data

In [23]:
## downloading the data 
downloadData()

kaggle.json
  0% 0.00/3.02M [00:00<?, ?B/s]
100% 3.02M/3.02M [00:00<00:00, 48.6MB/s]
  0% 0.00/17.2k [00:00<?, ?B/s]
100% 17.2k/17.2k [00:00<00:00, 15.2MB/s]
  0% 0.00/846k [00:00<?, ?B/s]
100% 846k/846k [00:00<00:00, 110MB/s]
  0% 0.00/17.2k [00:00<?, ?B/s]
100% 17.2k/17.2k [00:00<00:00, 14.4MB/s]
 98% 535M/548M [00:04<00:00, 143MB/s]
100% 548M/548M [00:04<00:00, 132MB/s]
 75% 37.0M/49.4M [00:00<00:00, 56.7MB/s]
100% 49.4M/49.4M [00:00<00:00, 90.7MB/s]
 71% 9.00M/12.7M [00:01<00:00, 11.2MB/s]
100% 12.7M/12.7M [00:01<00:00, 10.9MB/s]
  0% 0.00/1.13M [00:00<?, ?B/s]
100% 1.13M/1.13M [00:00<00:00, 160MB/s]
Archive:  /content/historical_transactions.csv.zip
  inflating: historical_transactions.csv  
Archive:  /content/new_merchant_transactions.csv.zip
  inflating: new_merchant_transactions.csv  
Archive:  /content/merchants.csv.zip
  inflating: merchants.csv           
Archive:  /content/train.csv.zip
  inflating: train.csv               
Archive:  /content/test.csv.zip
  inflating: test.

## Loading the Data

To get the transactions data and merchants for the card_id for which we need to predcit the loyalty score, we are gonna load the data in the notebook. But in terms of deployment ofthe model we will get from a database.

In [24]:
train_df,historical_transactions_df,new_merchant_transactions_df,test_df = loadData()

target  = train_df[['card_id','target']]
target.set_index('card_id', inplace =True)

cards = pd.concat([train_df.drop(['target'] , axis= 1) , test_df] , axis = 0)

## Predicting Loyalty Score for a Single Card_id

In [25]:
X = train_df.sample(1)['card_id'].to_list()

In [26]:
%%time
predict_loyalty_score(X)

Fetching the transactional and card_id data
PreProcess the transactions data......
Feature Engineering the transactions data.....
preprocess the Feature Engineered Data
Predicting the Loyalty Score.....
CPU times: user 2.59 s, sys: 64.8 ms, total: 2.66 s
Wall time: 6.09 s


,predicted_target
card_id,
C_ID_0b92e83d26,-0.977876


In [27]:
%%time
cal_rmse_score = rmse_score(X , target.loc[target.index.isin(X)])

Fetching the transactional and card_id data
PreProcess the transactions data......
Feature Engineering the transactions data.....
preprocess the Feature Engineered Data
Predicting the Loyalty Score.....
RMSE Score: 0.16609845708631488
CPU times: user 1.96 s, sys: 21.5 ms, total: 1.98 s
Wall time: 1.91 s


## Predicting Loyalty Score for a Set of train Card_id's

In [28]:
X = train_df.sample(1000)['card_id'].to_list()

In [29]:
%%time
predict_loyalty_score(X)

Fetching the transactional and card_id data
PreProcess the transactions data......
Feature Engineering the transactions data.....
preprocess the Feature Engineered Data
Predicting the Loyalty Score.....
CPU times: user 5.47 s, sys: 154 ms, total: 5.62 s
Wall time: 6.02 s


,predicted_target
card_id,
C_ID_f478df16fd,-0.916580
C_ID_642f591402,0.312313
C_ID_028bd7dc53,-3.748715
C_ID_78bb01b56e,-3.138297
C_ID_4cfc17c2fb,0.691464
...,...
C_ID_86ee1869c8,-0.671134
C_ID_17d7a77690,0.390104
C_ID_16e43043a3,-0.208643


In [30]:
%%time
cal_rmse_score = rmse_score(X , target.loc[target.index.isin(X)])

Fetching the transactional and card_id data
PreProcess the transactions data......
Feature Engineering the transactions data.....
preprocess the Feature Engineered Data
Predicting the Loyalty Score.....
RMSE Score: 4.522284770915642
CPU times: user 5.3 s, sys: 143 ms, total: 5.44 s
Wall time: 5.12 s


## Predicting Loyalty Score for Test Data

In [31]:
X = test_df.sample(1)['card_id'].to_list()

In [32]:
%%time
predict_loyalty_score(X)

Fetching the transactional and card_id data
PreProcess the transactions data......
Feature Engineering the transactions data.....
preprocess the Feature Engineered Data
Predicting the Loyalty Score.....
CPU times: user 1.83 s, sys: 14.6 ms, total: 1.85 s
Wall time: 1.77 s


,predicted_target
card_id,
C_ID_c5fe25fbfd,-2.82573


The Prediction time is low as 1.87 seconds. With, use of database to fetch the features of the transactions and card_id's the predcition time can be reduced more.